In [3]:
import sys
import statistic_lib as st
import numpy as np
import pandas as pd
import csv

In [4]:
df = pd.read_csv('../datasets/dataset_train.csv')

df.dropna(inplace=True)

df_numeric = df.select_dtypes(include=['number']).drop(columns=['Index', 'Care of Magical Creatures', 'Arithmancy', 'Astronomy'])

In [5]:
nb_exemples = len(df_numeric)

df_houses = df['Hogwarts House']
houses_list = df_houses.unique().tolist()
nb_houses = len(houses_list)

In [6]:
Y = np.zeros((nb_exemples, nb_houses))

classe_to_index = {classe: index for index, classe in enumerate(houses_list)}

for i, house in enumerate(df_houses):
    Y[i, classe_to_index[house]] = 1

In [7]:
features = df_numeric.columns.tolist()
nb_features = len(features)
features.append(('Bias'))

W = pd.DataFrame(columns=houses_list, index=features)
W = W.infer_objects(copy=False).fillna(0)
W

,Ravenclaw,Slytherin,Gryffindor,Hufflepuff
Herbology,0.0,0.0,0.0,0.0
Defense Against the Dark Arts,0.0,0.0,0.0,0.0
Divination,0.0,0.0,0.0,0.0
Muggle Studies,0.0,0.0,0.0,0.0
Ancient Runes,0.0,0.0,0.0,0.0
History of Magic,0.0,0.0,0.0,0.0
Transfiguration,0.0,0.0,0.0,0.0
Potions,0.0,0.0,0.0,0.0
Charms,0.0,0.0,0.0,0.0
Flying,0.0,0.0,0.0,0.0


In [8]:
X = np.array((df_numeric.apply(lambda x: (x - x.mean()) / x.std())))

In [9]:
ones = np.ones(X.shape[0])
X = np.column_stack((X,ones))
X

array([[ 0.86877347,  1.02575399,  0.3668627 , ...,  1.19808724,
        -0.51147256,  1.        ],
       [-1.37405992,  1.14894221, -2.14688607, ..., -1.00774958,
        -1.39695141,  1.        ],
       [ 1.25127087,  0.79192947,  0.71179165, ...,  1.81824066,
         0.07478891,  1.        ],
       ...,
       [-0.84101312, -0.95507478,  0.69354787, ..., -0.94547031,
         1.79388227,  1.        ],
       [ 0.43142147, -0.78165433,  0.85725534, ..., -0.32592323,
        -1.02213689,  1.        ],
       [ 0.81016221, -1.23320925,  0.82198404, ..., -0.07905294,
        -0.79667531,  1.        ]])

### Fonction Sigmoid

In [10]:
def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

### Descente de gradient

In [23]:
def gradient_descent(df_houses, house_name, W, W_house, X, Y, learning_rate=0.01, num_iterations=50000):
    current_house = np.where(df_houses == house_name, 1, 0)

    for i in range(num_iterations):
        Z = np.dot(X, W_house)
        Y_hat = sigmoid(Z)
        # print(len(Y_hat)) 1251 colonnes (1251 eleves), chaque scalaire represente la proba que l'eleve fasse partie de telle ou telle classe), 
        # et on loop en fonction du nb_iter et on fait ca pour chaque classe
        # au debut on a que des 0.5 car x = 0 donc aucune 'classe' n'est privilegiee
        gradient = np.dot(X.T, (Y_hat - current_house)) / current_house.shape[0]
        W_house = W_house - learning_rate * gradient

    print(f'gradient descent of {house_name} finish')
    return W_house

In [24]:
for house in houses_list:
    W[house] = gradient_descent(df_houses, house, W, W[house], X, Y, learning_rate=0.01, num_iterations=10000)

gradient descent of Ravenclaw finish
gradient descent of Slytherin finish
gradient descent of Gryffindor finish
gradient descent of Hufflepuff finish


L’algorithme One-Versus-All (One vs All) permet d’utiliser Logistic Regression pour la classification multi-classes. 
Le principe est simple : Il consiste à découper le problème de classification multi-classes en une multitude de problèmes de classification binaires.

In [82]:
with open('result_train_weights.csv', 'w', newline='') as file:
    W.to_csv(file)